## CBoW मॉडेल प्रशिक्षण

हे नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) चा एक भाग आहे.

या उदाहरणात, आपण CBoW भाषा मॉडेल प्रशिक्षण देऊन आपले स्वतःचे Word2Vec एम्बेडिंग स्पेस तयार करण्याचा विचार करू. आम्ही AG News डेटासेट मजकुराचा स्रोत म्हणून वापरू.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

प्रथम आपण आपला डेटासेट लोड करूया आणि टोकनायझर आणि शब्दसंग्रह परिभाषित करूया. आपण `vocab_size` 5000 वर सेट करू जेणेकरून गणनांवर थोडी मर्यादा येईल.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW मॉडेल

CBoW $2N$ शेजारील शब्दांवर आधारित एखादा शब्द अंदाज करण्यासाठी शिकतो. उदाहरणार्थ, जेव्हा $N=1$ असेल, तेव्हा वाक्य *I like to train networks* मधून आपल्याला खालील जोड्या मिळतील: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). येथे, पहिला शब्द हा शेजारील शब्द आहे जो इनपुट म्हणून वापरला जातो, आणि दुसरा शब्द हा आपण अंदाज करत असलेला शब्द आहे.

पुढील शब्दाचा अंदाज लावण्यासाठी नेटवर्क तयार करण्यासाठी, आपल्याला शेजारील शब्द इनपुट म्हणून द्यावा लागेल, आणि शब्द क्रमांक आउटपुट म्हणून मिळवावा लागेल. CBoW नेटवर्कची रचना खालीलप्रमाणे आहे:

* इनपुट शब्द एम्बेडिंग लेयरमधून पास केला जातो. हाच एम्बेडिंग लेयर आपला Word2Vec एम्बेडिंग असेल, त्यामुळे आपण त्याला `embedder` नावाच्या व्हेरिएबलमध्ये स्वतंत्रपणे परिभाषित करू. या उदाहरणात आपण एम्बेडिंग साइज = 30 वापरणार आहोत, जरी तुम्हाला उच्च डिमेन्शन्ससह प्रयोग करायचा असेल (खऱ्या Word2Vec मध्ये 300 असतो).
* एम्बेडिंग व्हेक्टर नंतर एका लीनियर लेयरमधून पास केला जाईल जो आउटपुट शब्दाचा अंदाज लावेल. त्यामुळे त्यात `vocab_size` न्यूरॉन्स असतील.

आउटपुटसाठी, जर आपण `CrossEntropyLoss` ला लॉस फंक्शन म्हणून वापरले, तर आपल्याला अपेक्षित परिणाम म्हणून फक्त शब्द क्रमांक द्यावे लागतील, वन-हॉट एन्कोडिंगशिवाय.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## प्रशिक्षण डेटा तयार करणे

आता आपण मुख्य फंक्शन प्रोग्राम करूया, जे मजकूरातून CBoW शब्द जोड्या तयार करेल. हे फंक्शन आपल्याला विंडो आकार निर्दिष्ट करण्याची परवानगी देईल आणि इनपुट व आउटपुट शब्दांच्या जोड्यांचा संच परत करेल. लक्षात ठेवा की हे फंक्शन शब्दांवर तसेच वेक्टर/टेंसरवर देखील वापरले जाऊ शकते - ज्यामुळे आपण मजकूर एन्कोड करू शकतो, `to_cbow` फंक्शनला पास करण्यापूर्वी.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


चला प्रशिक्षण डेटासेट तयार करूया. आपण सर्व बातम्या पाहू, `to_cbow` कॉल करून शब्द जोड्यांची यादी मिळवू, आणि त्या जोड्या `X` आणि `Y` मध्ये जोडू. वेळेच्या बचतीसाठी, आपण फक्त पहिल्या 10k बातम्या विचारात घेऊ - तुमच्याकडे जास्त वेळ असल्यास आणि चांगले एम्बेडिंग्ज मिळवायचे असल्यास तुम्ही ही मर्यादा सहज काढून टाकू शकता :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

आम्ही त्या डेटाला एका डेटासेटमध्ये रूपांतरित करू आणि डाटालोडर तयार करू:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

आम्ही त्या डेटाला एक डेटासेटमध्ये रूपांतरित करू आणि डेटालोडर तयार करू:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

आता आपण प्रत्यक्ष प्रशिक्षण करूया. आपण `SGD` ऑप्टिमायझर उच्च शिक्षण दरासह वापरू. तुम्ही `Adam` सारख्या इतर ऑप्टिमायझरचा वापर करूनही पाहू शकता. सुरुवातीला आपण 10 युगांसाठी प्रशिक्षण घेऊ - आणि जर तुम्हाला आणखी कमी तोटा हवा असेल तर तुम्ही ही सेल पुन्हा चालवू शकता.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec वापरण्याचा प्रयत्न

Word2Vec वापरण्यासाठी, आपल्या शब्दसंग्रहातील सर्व शब्दांसाठी संबंधित व्हेक्टर काढूया:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


वर्ड2व्हेकचा उपयोग समानार्थी शब्द शोधण्यासाठी करणे मनोरंजक आहे. खालील फंक्शन दिलेल्या इनपुटसाठी `n` जवळचे शब्द परत करेल. त्यांना शोधण्यासाठी, आपण $|w_i - v|$ चे नॉर्म मोजतो, जिथे $v$ हा आपल्या इनपुट शब्दाशी संबंधित व्हेक्टर आहे, आणि $w_i$ हा शब्दसंग्रहातील `i`-व्या शब्दाचे एन्कोडिंग आहे. त्यानंतर आम्ही अॅरे सॉर्ट करतो आणि `argsort` चा वापर करून संबंधित निर्देशांक परत करतो, आणि यादीतील पहिले `n` घटक घेतो, जे शब्दसंग्रहातील जवळच्या शब्दांच्या स्थानांचे एन्कोडिंग करतात.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## मुख्य मुद्दा

CBoW सारख्या हुशार तंत्रांचा वापर करून आपण Word2Vec मॉडेल प्रशिक्षित करू शकतो. तुम्ही skip-gram मॉडेल देखील प्रशिक्षित करण्याचा प्रयत्न करू शकता, जे मध्यवर्ती शब्द दिल्यास शेजारील शब्दाचा अंदाज लावण्यासाठी प्रशिक्षित केले जाते, आणि ते किती चांगले कार्य करते ते पाहू शकता.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
